**101 neural network architectures you need to learn**

This series of notebooks will walk through important neural network architectures. We'll start with the [`pytorch/examples`](https://github.com/pytorch/examples) directory and modify the networks to work with arbitrarily simple data. Let's get into it!

**IMAGE CLASSIFICATION NETWORKS**

**1. LeNet-5**

`examples/mnist` seems like a good place to start. Isn't `mnist` the "hello world" of machine learning or something?

If we examine `mnist/main.py`, we see a `Net` class that inherits from `nn.Module`. As it turns out, this is a slight modification of the [LeNet-5](http://yann.lecun.com/exdb/publis/pdf/lecun-01a.pdf) network (LeCun et al. 1998). We can find an implementation of LeNet-5 in the `kuangliu/pytorch-cifar/models` [repository](https://github.com/kuangliu/pytorch-cifar/tree/master/models). Let's navigate in that repository to `lenet.py`. First we'll do some imports: `torch` (PyTorch), `torch.nn`, and `torch.nn.functional` (the latter two are modules that define the operations we'll need to train this network and others):

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

This version of LeNet-5 is designed for CIFAR data, which are 32x32x3 RGB images. For other data, we may need to modify the input layer (e.g. one input channel for grayscale images, padding for images of different height and width) and output layer (where the number of output channels should match the number of classes). Let's take a look:

In [2]:
class LeNet(nn.Module):
    def __init__(self):
        super(LeNet, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1   = nn.Linear(16*5*5, 120)
        self.fc2   = nn.Linear(120, 84)
        self.fc3   = nn.Linear(84, 10)

    def forward(self, x):
        out = F.relu(self.conv1(x))
        out = F.max_pool2d(out, 2)
        out = F.relu(self.conv2(out))
        out = F.max_pool2d(out, 2)
        out = out.view(out.size(0), -1)
        out = F.relu(self.fc1(out))
        out = F.relu(self.fc2(out))
        out = self.fc3(out)
        return out

We won't spend too much time defining operations and their implementation - these are topics for another notebook. Let's just list what we're doing:

1) 2D convolution (3 input channels, 6 output channels, 5x5 kernels, default stride of 1) <br/>
2) ReLU activation <br/>
3) 2D max pooling (2x2 kernels) <br/>

4) 2D convolution (6 inputs, 16 outputs, and 5x5 kernels, default stride of 1) <br/>
5) ReLU activation <br/>
6) 2D max pooling (2x2 kernels) <br/>
7) Reshaping of tensor (3D to 1D) <br/>

8) Linear transformation with bias (400 inputs, 120 outputs) <br/>
9) ReLU activation <br/>
10) Linear transformation with bias (120 inputs, 84 outputs) <br/>
11) ReLU activation <br/>
12) Linear transformation with bias (84 inputs, 10 outputs)

We also need to backpropagate (update parameters through gradient descent), but we'll keep our focus on the architecture.

How many parameters do we learn? By default, convolutions are computed with bias but linear transformations are computed without bias (hence the +1). We don't learn any new parameters through activations, pooling, or reshaping, though of course there is a computational cost to applying these functions.

1) 6 outputs x 5x5 kernels = 150 parameters <br/>
4) 16 outputs x 5x5 kernels = 400 parameters <br/>
8) 400+1 inputs x 120 outputs = 48,120 parameters <br/>
10) 120+1 inputs x 84 outputs = 10,164 parameters <br/>
12) 84+1 inputs x 10 outputs = 850 parameters

Total = 59,684 parameters

Let's experiment with training LeNet-5 on MNIST. We'll take the `mnist/main.py` file, swap in `LeNet` for `Net`, and modify `nn.Conv2d` in `self.conv1` to take one input channel. We also need to keep the name of our class as `Net` (or change `Net` to `LeNet` elsewhere in the script). This also means we have to change `super(LeNet, self)` to `super(Net, self)`. We also need to pad our input images to change their dimensions from 28x28 to 32x32. A simple solution is to add `padding=2` to our first `nn.Conv2d`, which adds two rows and columns of zeroes to each size of the image. Finally, we need to change our `return` statement to apply `F.log_softmax(x, dim=1)`. (If we don't do this, our loss rapidly decreases without bound to `nan`, and the network fails to train.)

Let's train the network for 10 epochs with default hyperparameters and record the accuracy at each epoch. We have 10,000 observations in our test set, so we can multiply our accuracy by 10,000 to get the number of correct observations. We'll also log the increase in accuracy over the prior epoch for each epoch.

Epoch 1: 0.9457 <br/>
Epoch 2: 0.9736 (+0.279) <br/>
Epoch 3: 0.9758 (+0.022) <br/>
Epoch 4: 0.9793 (+0.035) <br/>
Epoch 5: 0.9826 (+0.033) <br/>
Epoch 6: 0.9864 (+0.038) <br/>
Epoch 7: 0.9862 (-0.002) <br/>
Epoch 8: 0.9861 (-0.001) <br/>
Epoch 9: 0.9840 (-0.021) <br/>
Epoch 10: **0.9872** (+0.032)

It appears that we're converging to the best accuracy we can get with this architecture and set of hyperparameters. Since we still got our best accuracy after 10 epochs, let's try it again for 20 epochs and record our results. If we train for 10 epochs again, we actually get the same accuracy for each epoch as when we trained the first time. (Actually, it's the same loss for each batch.) Is that a property of our network? Let's not worry about that for now, but it's interesting to note.

Epoch 11: 0.9881 <br/>
Epoch 12: 0.9890 <br/>
Epoch 13: 0.9893 <br/>
Epoch 14: 0.9895 <br/>
Epoch 15: 0.9894 <br/>
Epoch 16: 0.9899 <br/>
Epoch 17: **0.9910** <br/>
Epoch 18: 0.9895 <br/>
Epoch 19: 0.9882 <br/>
Epoch 20: 0.9900

Cool. There's a lot more experimentation we can do, but we should come up with a game plan for the other 100 networks to master. We'll keep going with the PyTorch `examples`. The next logical stop is `ImageNet`. The `main.py` file there implements a number of models (presumably these are all for image classification on the 1000 class ImageNet dataset), but the models themselves are not defined in the `ImageNet` directory. Where can we find them?

There's also a `pytorch/vision` [repository](https://github.com/pytorch/vision) imported as the Python `torchvision` package. If we `cd` into `torchvision/models`, we see a bunch of image classification architectures. Counting up from 1. LeNet-5, we have:

2. AlexNet <br/>
3. DenseNet <br/>
4. GoogLeNet <br/>
5. Inception <br/>
6. MNASNet <br/>
7. MobileNet <br/>
8. ResNet <br/>
9. ShuffleNetV2 <br/>
10. SqueezeNet <br/>
11. VGG

We also see three directories in `vision/torchvision/models`: `detection`, `segmentation`, and `video`. Let's plan to move on to `detection` after we've explored our image classification networks. Here we see two object detection networks, along with a bunch of other `.py` files:

12. Faster R-CNN <br/>
13. Mask R-CNN

`models/segmentation` has two semantic segmentation networks:

14. DeepLabV3 <br/>
15. FCN (Fully Convolutional Network)

`models/video` includes (16.) 3D-ResNet

In addition to `vision`, PyTorch has a `text` library for NLP, `audio` for audio signal processing, `fairseq` for sequence-to-sequence modeling. We also have some models left in the `examples` directory.

`text` is primarily datasets and files for loading and preprocessing data. There is a generic sentiment analysis model in `text`, and there is also a generic RNN in `examples/word_language_model`. Let's steer clear of generic models. Fortunately, `fairseq` has state-of-the-art implementations. Let's look inside.

In `fairseq`, we see both `examples` and `fairseq/models` directories. Let's start with the latter. Ignoring `roberta` (at least for now), we see a bunch of `.py` files in here. We'll want to take a closer look at some of these networks.

Let's summarize these networks without getting into the more specific implementation, which we'll cover elsewhere

**LeNet-5**

Conv2d - ReLU - MaxPool2d <br/>
Conv2d - ReLU - MaxPool2d <br/>
Flatten - Linear - ReLU <br/>
Linear - ReLU <br/>
Linear